In [53]:
import numpy as np
from scipy.sparse.linalg import eigsh
import scipy.sparse as sparse
from scipy.linalg import eigh
import scipy.linalg
import matplotlib.pyplot as plt
from qutip import *
import math

In [54]:
#plt.rcParams['figure.figsize']=8.5,5.1
plt.rcParams['font.size']=20
plt.rcParams['lines.linewidth']=1.5
plt.rcParams['text.usetex']=True

In [55]:
#Construction of necessary elements of theta and phi Hilbert spaces

# For phi

N=50
a_phi=np.diag(np.sqrt(np.arange(1, N+1)), 1)
ad_phi=np.diag(np.sqrt(np.arange(1, N+1)), -1)
N_phi=ad_phi@a_phi
Id_N=np.eye(N+1)

# For theta

M=20
Id_M=np.eye(2*M+1)
n_theta=np.flipud(np.fliplr(np.diag(np.arange(-M,M+1), 0)))

#Values of equilibrium

phi_ext_val=np.pi
ng_val=0.5

#Qubit parameters 1 

E_Cs=1.0
E_Cj=1.0
E_L=1.0
E_J=1.0
alpha=(1j*(E_Cj/E_L)**(1.0/4.0))

# Qubit parameters 2

E_Cs_2=(1.0/20)
E_Cj_2=1.0
E_L_2=(1.0/16)
E_J_2=(1.0/5.0)
alpha2=(1j*(E_Cj_2/E_L_2)**(1.0/4.0))

#FOR EXPECTATION VALUES

#PHI
print("PHI")
print((E_Cj/E_L)**(1.0/4.0)*(a_phi+ad_phi))
phi=np.kron(Id_M,((E_Cj/E_L)**(1.0/4.0))*(a_phi+ad_phi))
print("kron PHI")
print(phi)
phi2=np.kron(Id_M,((E_Cj_2/E_L_2)**(1.0/4.0))*(a_phi+ad_phi))

#COS_THETA
costheta=np.diag(np.ones(2*M),1)+np.diag(np.ones(2*M),-1) 
costheta[0][2*M]+=1
costheta[2*M][0]+=1
print(costheta)
print("COS")
cos_theta=np.kron(costheta,Id_N)
print("kron COS")
print(cos_theta)

#N_PHI
N_phi_extended=np.kron(Id_M,N_phi)

#N_theta
n_theta_extended=np.kron(n_theta,Id_N)


#FOR SAVING VALUES OF CALCULATIONS

#1ST VALUES

ng1=np.arange(0.495,0.506,0.01)
print(ng1)
a=np.pi-(np.pi*0.01)
b=np.pi+(np.pi*0.01 + 0.001)
phi_ext1=np.arange(a,b,(0.01*np.pi)*2)
print(phi_ext1)
Energies=[]
Eigenvectors=[]

Expvals_costheta_NG=[]
Expvals_phi_NG=[]
Expvals_N_phi_NG=[]
Expvals_n_theta_NG=[]

Expvals_costheta_PHIEXT=[]
Expvals_phi_PHIEXT=[]
Expvals_N_phi_PHIEXT=[]
Expvals_n_theta_PHIEXT=[]

Overlaps_NG=[]
Overlaps_PHIEXT=[]

EIGENVECTS_NG=[]
EIGENVECTS_PHIEXT=[]

Overlaps_UP_NG=[]
Overlaps_UP_PHIEXT=[]
Overlaps_SIDE_NG=[]
Overlaps_SIDE_PHIEXT=[]
Overlaps_CROSSED_NG=[]
Overlaps_CROSSED_PHIEXT=[]

#2ND VALUES

ng2=np.arange(0.495,0.506,0.01)
phi_ext2=np.arange(a,b,(0.01*np.pi)*2)
Energies2=[]
Eigenvectors2=[]

Expvals2_costheta_NG=[]
Expvals2_phi_NG=[]
Expvals2_N_phi_NG=[]
Expvals2_n_theta_NG=[]

Expvals2_costheta_PHIEXT=[]
Expvals2_phi_PHIEXT=[]
Expvals2_N_phi_PHIEXT=[]
Expvals2_n_theta_PHIEXT=[]

Overlaps2_NG=[]
Overlaps2_PHIEXT=[]

EIGENVECTS2_NG=[]
EIGENVECTS2_PHIEXT=[]

Overlaps2_UP_NG=[]
Overlaps2_UP_PHIEXT=[]
Overlaps2_SIDE_NG=[]
Overlaps2_SIDE_PHIEXT=[]
Overlaps2_CROSSED_NG=[]
Overlaps2_CROSSED_PHIEXT=[]


PHI
[[0.         1.         0.         ... 0.         0.         0.        ]
 [1.         0.         1.41421356 ... 0.         0.         0.        ]
 [0.         1.41421356 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         7.         0.        ]
 [0.         0.         0.         ... 7.         0.         7.07106781]
 [0.         0.         0.         ... 0.         7.07106781 0.        ]]
kron PHI
[[0.         1.         0.         ... 0.         0.         0.        ]
 [1.         0.         1.41421356 ... 0.         0.         0.        ]
 [0.         1.41421356 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         7.         0.        ]
 [0.         0.         0.         ... 7.         0.         7.07106781]
 [0.         0.         0.         ... 0.         7.07106781 0.        ]]
[[0. 1. 0. ... 0. 0. 1.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1

In [56]:
# Construction of the 0-pi Hamiltonian

def Hamiltonian_0pi(EJ,ECS,ECJ,EL,n,phi,alph):
    #Initial
    H=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Free Hamiltonian construction
    H1=4*ECS*((n_theta-n*Id_M)@(n_theta-n*Id_M))
    H_theta=np.kron(H1,Id_N)
    H2=2*np.sqrt(ECJ*EL)*(N_phi+0.5*Id_N)
    H_phi=np.kron(Id_M,H2)
    H_0=H_theta+H_phi
    #H_0=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Interaction Hamiltonian construction
    cos_theta_ext=np.diag(np.ones(2*M),1)+np.diag(np.ones(2*M),-1) 
    #cos_theta=np.diag(np.ones(2*M-1),2)+np.diag(np.ones(2*M-1),-2)
    cos_theta_ext[0][2*M]+=1
    cos_theta_ext[2*M][0]+=1
    #cos_phi=(0.5*((displace(N+1,alph)*(math.e**(-1j*phi/2)))+(displace(N+1,-alph)*(math.e**(1j*phi/2)))))*np.diag(np.ones(N+1),0)
    cos_phi=0.5*(scipy.linalg.expm(alph*(a_phi+ad_phi)-(Id_N*1j*phi/2))+scipy.linalg.expm(-alph*(a_phi+ad_phi)+(Id_N*1j*phi/2)))
    H_I=EJ*np.kron(cos_theta_ext,cos_phi.real)#SIN EL2 
    #H_I=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Total Hamiltonian
    H+=H_0-H_I
    return H

In [57]:
def calculate_expval(vect,operator):
    return np.conj(vect)@operator@vect

In [58]:
def calculate_overlap(vect1,vect2):
    return np.abs(np.conj(vect1)@vect2)**2

In [59]:
Vprueba=[1,1,0]
Vprueba2=[0,2,0]
print(Vprueba)
O=calculate_overlap(Vprueba,Vprueba2)
print("O=",O)
matrix=[[0,1,0],[1,0,1],[0,1,0]]
print(matrix)
a=calculate_expval(Vprueba,matrix)
print(a)


[1, 1, 0]
O= 4
[[0, 1, 0], [1, 0, 1], [0, 1, 0]]
2


In [60]:
Vs=[]
V1=np.array([1,1,0])
V2=np.array([0,2,0])
V3=np.array([1,1,1])
Vs.extend([V1,V2,V3])
print(Vs)
print(Vs[0])
print(calculate_overlap(Vs[0],Vs[1]))


[array([1, 1, 0]), array([0, 2, 0]), array([1, 1, 1])]
[1 1 0]
4


## $\text{1. Plot for } E_{Cs} = E_{Cj} = E_L = 1$, $\varphi_{ext}= \pi$, $\alpha = i \left(\frac{E_{CJ}}{E_L}\right)^{1/4} = i$.

In [61]:
# Calculations 1

for n1 in ng1:
    print("Step for ng = ",n1)
    H1=Hamiltonian_0pi(E_J,E_Cs,E_Cj,E_L,n1,phi_ext_val,alpha)
    print("Hamiltonian built, len(H)=",len(H1)**2)
    #vals,vects=np.linalg.eigh(H)
    valsNG, vectsNG = eigsh(H1, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",valsNG[0]," and ",valsNG[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vectsNG[:,0]," and ",vectsNG[:,1])
    EIGENVECTS_NG.extend([vectsNG[:,0],vectsNG[:,1]])
    print(EIGENVECTS_NG)
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval_costheta_0=calculate_expval(vectsNG[:,0],cos_theta)
    Expval_costheta_1=calculate_expval(vectsNG[:,1],cos_theta)
    #Phi
    Expval_phi_0=calculate_expval(vectsNG[:,0],phi)
    Expval_phi_1=calculate_expval(vectsNG[:,1],phi)
    #N_phi
    Expval_N_phi_0=calculate_expval(vectsNG[:,0],N_phi_extended)
    Expval_N_phi_1=calculate_expval(vectsNG[:,1],N_phi_extended)
    #n_theta
    Expval_n_theta_0=calculate_expval(vectsNG[:,0],n_theta_extended)
    Expval_n_theta_1=calculate_expval(vectsNG[:,1],n_theta_extended)
    #Overlap between the states of the same value of ng
    Overlap=calculate_overlap(vectsNG[:,0],vectsNG[:,1])
    print("Overlap = ",Overlap)
    #TOTALS
    Overlaps_NG.append(Overlap)
    Expvals_costheta_NG.extend([Expval_costheta_0,Expval_costheta_1])
    Expvals_phi_NG.extend([Expval_phi_0,Expval_phi_1])
    Expvals_N_phi_NG.extend([Expval_N_phi_0,Expval_N_phi_1])
    Expvals_n_theta_NG.extend([Expval_n_theta_0,Expval_n_theta_1])

Step for ng =  0.495
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.7550311282880182  and  1.788925914299877
The eigenvectors of the GS and the 1st excited state are  [-5.73771836e-18  4.21393650e-19 -5.04124093e-19 ...  8.97175961e-18
  7.11844053e-17 -8.87696242e-19]  and  [ 2.72680230e-17 -5.31473219e-18  8.13008269e-18 ...  3.43768396e-17
 -1.18182315e-16 -2.21764456e-16]
[array([-5.73771836e-18,  4.21393650e-19, -5.04124093e-19, ...,
        8.97175961e-18,  7.11844053e-17, -8.87696242e-19]), array([ 2.72680230e-17, -5.31473219e-18,  8.13008269e-18, ...,
        3.43768396e-17, -1.18182315e-16, -2.21764456e-16])]
Eigenvalues and eigenvectors calculated
Overlap =  1.8980830963398434e-29
Step for ng =  0.505
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.7550311282882096  and  1.7889259142999792
The eigenvectors of the GS and the 1st excited state are  [ 4.43477360e-18  6.48734000e-19  5.47

In [62]:
# Calculations 1

for phi1 in phi_ext1:
    print("Step for phi_ext = ",phi1)
    H2=Hamiltonian_0pi(E_J,E_Cs,E_Cj,E_L,ng_val,phi1,alpha)
    print("Hamiltonian built, len(H)=",len(H2)**2)
    #vals,vects=np.linalg.eigh(H)
    valsPHI, vectsPHI = eigsh(H2, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",valsPHI[0]," and ",valsPHI[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vectsPHI[:,0]," and ",vectsPHI[:,1])
    EIGENVECTS_PHIEXT.extend([vectsPHI[:,0],vectsPHI[:,1]])
    print(EIGENVECTS_PHIEXT)
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval_costheta_0=calculate_expval(vectsPHI[:,0],cos_theta)
    Expval_costheta_1=calculate_expval(vectsPHI[:,1],cos_theta)
    #Phi
    Expval_phi_0=calculate_expval(vectsPHI[:,0],phi)
    Expval_phi_1=calculate_expval(vectsPHI[:,1],phi)
    #N_phi
    Expval_N_phi_0=calculate_expval(vectsPHI[:,0],N_phi_extended)
    Expval_N_phi_1=calculate_expval(vectsPHI[:,1],N_phi_extended)
    #n_theta
    Expval_n_theta_0=calculate_expval(vectsPHI[:,0],n_theta_extended)
    Expval_n_theta_1=calculate_expval(vectsPHI[:,1],n_theta_extended)
    #Overlap between the states of the same value of ng
    Overlap=calculate_overlap(vectsPHI[:,0],vectsPHI[:,1])
    print("Overlap = ",Overlap)
    #TOTALS
    Overlaps_PHIEXT.append(Overlap)
    Expvals_costheta_PHIEXT.extend([Expval_costheta_0,Expval_costheta_1])
    Expvals_phi_PHIEXT.extend([Expval_phi_0,Expval_phi_1])
    Expvals_N_phi_PHIEXT.extend([Expval_N_phi_0,Expval_N_phi_1])
    Expvals_n_theta_PHIEXT.extend([Expval_n_theta_0,Expval_n_theta_1])

Step for phi_ext =  3.1101767270538954
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.763178252558364  and  1.7807346115217029
The eigenvectors of the GS and the 1st excited state are  [-3.58745780e-17 -5.38029281e-18  1.12870830e-17 ...  1.47058270e-17
  1.03973595e-19 -3.46154689e-17]  and  [ 1.38438333e-17  3.29631051e-18 -1.60871502e-18 ... -1.19227378e-17
 -8.73756199e-20  5.63150656e-17]
[array([-3.58745780e-17, -5.38029281e-18,  1.12870830e-17, ...,
        1.47058270e-17,  1.03973595e-19, -3.46154689e-17]), array([ 1.38438333e-17,  3.29631051e-18, -1.60871502e-18, ...,
       -1.19227378e-17, -8.73756199e-20,  5.63150656e-17])]
Eigenvalues and eigenvectors calculated
Overlap =  3.081487911019579e-33
Step for phi_ext =  3.1730085801256913
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.7631782525584991  and  1.7807346115218272
The eigenvectors of the GS and the 1st excited state are  [ 3

EXPECTATION VALUES FOR THE FIRST SET OF VALUES FOR THE ENERGIES

In [63]:
#VARYING NG
print("VARYING NG")
print("Exp values of cos_theta varying ng are ",Expvals_costheta_NG)
print("Exp values of phi varying ng are ",Expvals_phi_NG)
print("Exp values of N_phi varying ng are ",Expvals_N_phi_NG)
print("Exp values of n_theta varying ng are ",Expvals_n_theta_NG)
#VARYING PHI_EXT
print("VARYING PHI_EXT")
print("Exp values of cos_theta varying phi_ext are ",Expvals_costheta_PHIEXT)
print("Exp values of phi varying phi_ext are ",Expvals_phi_PHIEXT)
print("Exp values of N_phi varying phi_ext are ",Expvals_N_phi_PHIEXT)
print("Exp values of n_theta varying phi_ext are ",Expvals_n_theta_PHIEXT)

VARYING NG
Exp values of cos_theta varying ng are  [-8.765625670556048e-12, 8.74806964150756e-12, 1.1891435323497299e-12, -1.18738254481704e-12]
Exp values of phi varying ng are  [-4.550808497963032e-12, 4.408263014222102e-12, 6.03456508944049e-13, -5.929924778400095e-13]
Exp values of N_phi varying ng are  [0.08693140982194662, 0.09220055517257672, 0.08693140982192679, 0.09220055517258262]
Exp values of n_theta varying ng are  [0.0736982264955744, 0.9209886856432846, 0.9263017735044378, 0.07901131435672198]
VARYING PHI_EXT
Exp values of cos_theta varying phi_ext are  [1.0738891143108826, -1.0705380216291709, -1.0738891143108784, 1.070538021629147]
Exp values of phi varying phi_ext are  [0.5556498947237871, -0.5619624882128226, -0.5556498947237541, 0.5619624882127604]
Exp values of N_phi varying phi_ext are  [0.08873497723141506, 0.09026030865728715, 0.08873497723140274, 0.09026030865727122]
Exp values of n_theta varying phi_ext are  [0.49999999999894207, 0.500000000001031, 0.500000000

In [64]:
Overlaps_UP_NG.extend([calculate_overlap(EIGENVECTS_NG[0],EIGENVECTS_NG[1]),calculate_overlap(EIGENVECTS_NG[2],EIGENVECTS_NG[3])])
Overlaps_UP_PHIEXT.extend([calculate_overlap(EIGENVECTS_PHIEXT[0],EIGENVECTS_PHIEXT[1]),calculate_overlap(EIGENVECTS_PHIEXT[2],EIGENVECTS_PHIEXT[3])])
Overlaps_SIDE_NG.extend([calculate_overlap(EIGENVECTS_NG[0],EIGENVECTS_NG[2]),calculate_overlap(EIGENVECTS_NG[1],EIGENVECTS_NG[3])])
Overlaps_SIDE_PHIEXT.extend([calculate_overlap(EIGENVECTS_PHIEXT[0],EIGENVECTS_PHIEXT[2]),calculate_overlap(EIGENVECTS_PHIEXT[1],EIGENVECTS_PHIEXT[3])])
Overlaps_CROSSED_NG.extend([calculate_overlap(EIGENVECTS_NG[0],EIGENVECTS_NG[3]),calculate_overlap(EIGENVECTS_NG[1],EIGENVECTS_NG[2])])
Overlaps_CROSSED_PHIEXT.extend([calculate_overlap(EIGENVECTS_PHIEXT[0],EIGENVECTS_PHIEXT[3]),calculate_overlap(EIGENVECTS_PHIEXT[1],EIGENVECTS_PHIEXT[2])])
print(Overlaps_UP_NG)
print(Overlaps_UP_PHIEXT)
print(Overlaps_SIDE_NG)
print(Overlaps_SIDE_PHIEXT)
print(Overlaps_CROSSED_NG)
print(Overlaps_CROSSED_PHIEXT)

[1.8980830963398434e-29, 1.033811054640646e-30]
[3.081487911019579e-33, 1.4344804171891585e-64]
[1.2495403791427533e-23, 1.2452190007935233e-23]
[1.114147976822808e-25, 1.1111853110856374e-25]
[0.9999062295640778, 0.9999062295640816]
[0.9999750125015151, 0.9999750125015158]


## $\text{2. Plot for } E_{Cs} = \frac{1}{20}$, $E_{CJ} = 1$, $E_L = \frac{1}{16}$, $\varphi_{ext} = \pi$, $\alpha = i \left(\frac{E_{CJ}}{E_L}\right)^{1/4} = 2i$.

In [65]:
# Calculations 2

for n2 in ng2:
    print("Step for ng = ",n2)
    H3=Hamiltonian_0pi(E_J_2,E_Cs_2,E_Cj_2,E_L_2,n2,phi_ext_val,alpha2)
    print("Hamiltonian built, len(H)=",len(H3)**2)
    #vals2,vects2=np.linalg.eigh(H2)
    vals2NG, vects2NG = eigsh(H3, k=2, which='SA')
    #vals2,_=np.linalg.eigh(H3)
    print("The eigenvalues of the GS and the 1st excited state are ",vals2NG[0]," and ",vals2NG[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vects2NG[:,0]," and ",vects2NG[:,1])
    EIGENVECTS2_NG.extend([vects2NG[:,0],vects2NG[:,1]])
    print(EIGENVECTS2_NG)
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval2_costheta_0=calculate_expval(vects2NG[:,0],cos_theta)
    Expval2_costheta_1=calculate_expval(vects2NG[:,1],cos_theta)
    #Phi
    Expval2_phi_0=calculate_expval(vects2NG[:,0],phi)
    Expval2_phi_1=calculate_expval(vects2NG[:,1],phi)
    #N_phi
    Expval2_N_phi_0=calculate_expval(vects2NG[:,0],N_phi_extended)
    Expval2_N_phi_1=calculate_expval(vects2NG[:,1],N_phi_extended)
    #n_theta
    Expval2_n_theta_0=calculate_expval(vects2NG[:,0],n_theta_extended)
    Expval2_n_theta_1=calculate_expval(vects2NG[:,1],n_theta_extended)
    #Overlap between the states of the same value of ng
    Overlap2=calculate_overlap(vects2NG[:,0],vects2NG[:,1])
    print("Overlap = ",Overlap2)
    #TOTALS
    Overlaps2_NG.append(Overlap2)
    Expvals2_costheta_NG.extend([Expval2_costheta_0,Expval2_costheta_1])
    Expvals2_phi_NG.extend([Expval2_phi_0,Expval2_phi_1])
    Expvals2_N_phi_NG.extend([Expval2_N_phi_0,Expval2_N_phi_1])
    Expvals2_n_theta_NG.extend([Expval2_n_theta_0,Expval2_n_theta_1])

Step for ng =  0.495
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.27416445603158646  and  0.27611539062214574
The eigenvectors of the GS and the 1st excited state are  [ 3.13994167e-18 -8.76849843e-18  5.53310271e-18 ...  8.50818429e-17
 -6.86138594e-18  7.64303939e-17]  and  [ 2.03141150e-18 -4.38794573e-18 -1.44736451e-18 ... -6.33069432e-18
  5.88859710e-18  8.79424043e-17]
[array([ 3.13994167e-18, -8.76849843e-18,  5.53310271e-18, ...,
        8.50818429e-17, -6.86138594e-18,  7.64303939e-17]), array([ 2.03141150e-18, -4.38794573e-18, -1.44736451e-18, ...,
       -6.33069432e-18,  5.88859710e-18,  8.79424043e-17])]
Eigenvalues and eigenvectors calculated
Overlap =  3.691953814913924e-30
Step for ng =  0.505
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.2741644560315936  and  0.27611539062214796
The eigenvectors of the GS and the 1st excited state are  [-8.96736323e-18 -1.13667983e-17 -9

In [66]:
# Calculations 2

for phi2 in phi_ext2:
    print("Step for phi_ext = ",phi2)
    H4=Hamiltonian_0pi(E_J_2,E_Cs_2,E_Cj_2,E_L_2,ng_val,phi2,alpha2)
    print("Hamiltonian built, len(H)=",len(H4)**2)
    #vals,vects=np.linalg.eigh(H)
    vals2PHI, vects2PHI = eigsh(H4, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",vals2PHI[0]," and ",vals2PHI[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vects2PHI[:,0]," and ",vects2PHI[:,1])
    EIGENVECTS2_PHIEXT.extend([vects2PHI[:,0],vects2PHI[:,1]])
    print(EIGENVECTS2_PHIEXT)
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval2_costheta_0=calculate_expval(vects2PHI[:,0],cos_theta)
    Expval2_costheta_1=calculate_expval(vects2PHI[:,1],cos_theta)
    #Phi
    Expval2_phi_0=calculate_expval(vects2PHI[:,0],phi)
    Expval2_phi_1=calculate_expval(vects2PHI[:,1],phi)
    #N_phi
    Expval2_N_phi_0=calculate_expval(vects2PHI[:,0],N_phi_extended)
    Expval2_N_phi_1=calculate_expval(vects2PHI[:,1],N_phi_extended)
    #n_theta
    Expval2_n_theta_0=calculate_expval(vects2PHI[:,0],n_theta_extended)
    Expval2_n_theta_1=calculate_expval(vects2PHI[:,1],n_theta_extended)
    #Overlap between the states of the same value of ng
    Overlap2=calculate_overlap(vects2PHI[:,0],vects2PHI[:,1])
    print("Overlap = ",Overlap2)
    #TOTALS
    Overlaps2_PHIEXT.append(Overlap2)
    Expvals2_costheta_PHIEXT.extend([Expval2_costheta_0,Expval2_costheta_1])
    Expvals2_phi_PHIEXT.extend([Expval2_phi_0,Expval2_phi_1])
    Expvals2_N_phi_PHIEXT.extend([Expval2_N_phi_0,Expval2_N_phi_1])
    Expvals2_n_theta_PHIEXT.extend([Expval2_n_theta_0,Expval2_n_theta_1])

Step for phi_ext =  3.1101767270538954
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.27470737232474063  and  0.27556407700441987
The eigenvectors of the GS and the 1st excited state are  [ 4.72993947e-17  5.22868305e-18  1.02530643e-17 ...  5.82224684e-17
 -2.39773609e-17  5.13846277e-17]  and  [-8.42209680e-18 -1.19842472e-18 -1.52115684e-18 ...  2.21897548e-16
 -6.72333284e-17 -3.00306936e-16]
[array([ 4.72993947e-17,  5.22868305e-18,  1.02530643e-17, ...,
        5.82224684e-17, -2.39773609e-17,  5.13846277e-17]), array([-8.42209680e-18, -1.19842472e-18, -1.52115684e-18, ...,
        2.21897548e-16, -6.72333284e-17, -3.00306936e-16])]
Eigenvalues and eigenvectors calculated
Overlap =  7.703719777548943e-32
Step for phi_ext =  3.1730085801256913
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.2747073723247328  and  0.2755640770043898
The eigenvectors of the GS and the 1st excited state are  

EXPECTATION VALUES FOR THE SECOND SET OF VALUES FOR THE ENERGIES

In [67]:
#VARYING NG
print("VARYING NG")
print("Exp values of cos_theta varying ng are ",Expvals2_costheta_NG)
print("Exp values of phi varying ng are ",Expvals2_phi_NG)
print("Exp values of N_phi varying ng are ",Expvals2_N_phi_NG)
print("Exp values of n_theta varying ng are ",Expvals2_n_theta_NG)
#VARYING PHI_EXT
print("VARYING PHI_EXT")
print("Exp values of cos_theta varying phi_ext are ",Expvals2_costheta_PHIEXT)
print("Exp values of phi varying phi_ext are ",Expvals2_phi_PHIEXT)
print("Exp values of N_phi varying phi_ext are ",Expvals2_N_phi_PHIEXT)
print("Exp values of n_theta varying phi_ext are ",Expvals2_n_theta_PHIEXT)

VARYING NG
Exp values of cos_theta varying ng are  [8.694834648219604e-12, -8.72076131575369e-12, -1.0665545273552376e-12, 1.0765976715990578e-12]
Exp values of phi varying ng are  [1.7713885731809622e-12, -1.7714666342472132e-12, -2.1898848632283792e-13, 2.139509768992675e-13]
Exp values of N_phi varying ng are  [0.044023867412546526, 0.044248400525226064, 0.0440238674125472, 0.044248400525226744]
Exp values of n_theta varying ng are  [0.012071718350906889, 0.9875368300487424, 0.9879282816490931, 0.012463169951260428]
VARYING PHI_EXT
Exp values of cos_theta varying phi_ext are  [1.0837543531326994, -1.0800974073986658, -1.083754353132676, 1.0800974073986618]
Exp values of phi varying phi_ext are  [0.21752466351103222, -0.21875833128051825, -0.21752466351104943, 0.21875833128052044]
Exp values of N_phi varying phi_ext are  [0.04409616925676467, 0.04417101278913775, 0.044096169256767945, 0.04417101278913814]
Exp values of n_theta varying phi_ext are  [0.5000000000023181, 0.4999999999976

In [68]:
Overlaps2_UP_NG.extend([calculate_overlap(EIGENVECTS2_NG[0],EIGENVECTS2_NG[1]),calculate_overlap(EIGENVECTS2_NG[2],EIGENVECTS2_NG[3])])
Overlaps2_UP_PHIEXT.extend([calculate_overlap(EIGENVECTS2_PHIEXT[0],EIGENVECTS2_PHIEXT[1]),calculate_overlap(EIGENVECTS2_PHIEXT[2],EIGENVECTS2_PHIEXT[3])])
Overlaps2_SIDE_NG.extend([calculate_overlap(EIGENVECTS2_NG[0],EIGENVECTS2_NG[2]),calculate_overlap(EIGENVECTS2_NG[1],EIGENVECTS2_NG[3])])
Overlaps2_SIDE_PHIEXT.extend([calculate_overlap(EIGENVECTS2_PHIEXT[0],EIGENVECTS2_PHIEXT[2]),calculate_overlap(EIGENVECTS2_PHIEXT[1],EIGENVECTS2_PHIEXT[3])])
Overlaps2_CROSSED_NG.extend([calculate_overlap(EIGENVECTS2_NG[0],EIGENVECTS2_NG[3]),calculate_overlap(EIGENVECTS2_NG[1],EIGENVECTS2_NG[2])])
Overlaps2_CROSSED_PHIEXT.extend([calculate_overlap(EIGENVECTS2_PHIEXT[0],EIGENVECTS2_PHIEXT[3]),calculate_overlap(EIGENVECTS2_PHIEXT[1],EIGENVECTS2_PHIEXT[2])])
print(Overlaps2_UP_NG)
print(Overlaps2_UP_PHIEXT)
print(Overlaps2_SIDE_NG)
print(Overlaps2_SIDE_PHIEXT)
print(Overlaps2_CROSSED_NG)
print(Overlaps2_CROSSED_PHIEXT)

[3.691953814913924e-30, 1.3461813448158161e-30]
[7.703719777548943e-32, 3.0814879110195815e-33]
[1.2394969620822493e-23, 1.2389855016741999e-23]
[1.6696833012830422e-25, 1.6719523549063214e-25]
[0.9999983013475638, 0.9999983013475634]
[0.9999833013646552, 0.9999833013646552]
